## Importing libraries

In [1]:
#import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import re
#import time

import dask.dataframe as dd
#import dask.array as da
import dask.bag as db
import dask
from dask.distributed import Client, SSHCluster
import dask.distributed

import time

# import hvplot.dask

## Cluster up

In [2]:
cluster = SSHCluster(
            ["bhbh-1", 'bhbh-1', "bhbh-2", "bhbh-3", "bhbh-4", "bhbh-5"],
            connect_options={"client_keys": "/home/ubuntu/private/tbertola_key.pem"},
            worker_options={"n_workers": 4,
                            "nthreads": 4}, # because each bhbh-* has 4 cores
            scheduler_options={"port": 8786, "dashboard_address": ":8787"}
            )

2023-06-14 17:14:25,269 - distributed.deploy.ssh - INFO - 2023-06-14 17:14:25,268 - distributed.scheduler - INFO - State start
2023-06-14 17:14:25,272 - distributed.deploy.ssh - INFO - 2023-06-14 17:14:25,269 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-scratch-space/worker-naemo3jb', purging
2023-06-14 17:14:25,273 - distributed.deploy.ssh - INFO - 2023-06-14 17:14:25,270 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-scratch-space/worker-it31sb6m', purging
2023-06-14 17:14:25,280 - distributed.deploy.ssh - INFO - 2023-06-14 17:14:25,280 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-scratch-space/worker-k4zcapxk', purging
2023-06-14 17:14:25,285 - distributed.deploy.ssh - INFO - 2023-06-14 17:14:25,285 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-scratch-space/worker-_uangxs3', purging
2023-06-14 17:14:25,298 - distributed.deploy.ssh - INFO - 2023-06

In [3]:
client=Client(cluster)

/home/ubuntu/miniconda3/lib/python3.10/site-packages/distributed/client.py:1388: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+------------------+
| Package | Client | Scheduler | Workers          |
+---------+--------+-----------+------------------+
| tornado | 6.3.2  | 6.3.2     | {'6.3.2', '6.2'} |
+---------+--------+-----------+------------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


## Client 

In [4]:
client

<Client: 'tcp://10.67.22.140:8786' processes=16 threads=64, memory=124.21 GiB>

## Creating list of directories

In [5]:
import glob
dir_path = '/mnt/bhbh/fiducial_Hrad_5M/sevn_output_*'
dir_list = glob.glob(dir_path)
print(len(dir_list))

60


## Creating the bag of bags

In [6]:
bag_2=db.from_sequence([
                        [dir_ + f'/0/output_{thread}.csv', 
                         dir_ + f'/0/evolved_{thread}.dat',
                         dir_ + f'/0/logfile_{thread}.dat'] for dir_ in dir_list for thread in range(30)], npartitions=60*30)

## Pre-processing function for the bag_of_thread

In [7]:
# def preprocessing_bag_of_thread(bag_of_thread):  #bag of bag

def preprocessing_bag_of_thread(paths):
    
    output_column_to_read = ['name', 'Mass_0', 'RemnantType_0', 'Mass_1', 'RemnantType_1',
                         'Semimajor','Eccentricity','GWtime','BWorldtime']

    output_column_type = ['string', 'float64', 'int64', 'float64', 'int64',
                      'float64', 'float64', 'float64', 'float64', 'int64']

    evolved_column_to_read = ['name', 'Mass_0', 'Z_0', 'SN_0', 'Mass_1', 'SN_1', 'a', 'e']


    evolved_column_type = ['string', 'float64', 'float64', 'string', 'float64', 
                      'string', 'float64', 'float64']

    drop_list = ['RemnantType_0',  'RemnantType_1']
    
    # decomment if you want the bag of bag
   # paths = bag_of_thread.take(3)

   
    #preprocessing output
    
    output = pd.read_csv(paths[0], usecols=output_column_to_read, dtype=dict(zip(output_column_to_read, output_column_type))).\
                rename(columns={'Mass_0':'Mass_0_out', 'Mass_1':'Mass_1_out'})

    idxBHBH=(output.RemnantType_0==6) & (output.RemnantType_1==6) & (output.Semimajor.notnull())
    output=output[idxBHBH]
    
    df=output
    
    
    #preprocessing evolved
    
    alpha = float(re.findall(r".+(?<=A)(.*)(?=L)", paths[1])[0])
    
    evolved = pd.read_table(paths[1], sep='\s+', usecols=evolved_column_to_read, dtype=dict(zip(evolved_column_to_read, evolved_column_type)))                
    
    evolved['alpha'] = alpha
    
    #preprocessing logfile
    
    logfile = pd.read_csv(paths[2], header=None)

    df_RLO = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);RLO_BEGIN;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='RLO').\
                reset_index()

    df_CE = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);CE;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='CE').\
                reset_index()

    df_BSN = logfile[0].str.extract(r"B;((?:\d*\_)?\d+);(\d+);BSN;").\
                dropna().\
                rename(columns={0:'name', 1:'ID'}).\
                groupby(['name']).\
                size().to_frame(name='BSN').\
                reset_index()

    
    #merge
    bhbh = evolved.merge(output, on=['name'], how='inner').\
                   merge(df_RLO, on=['name'], how='left').\
                   merge(df_CE,  on=['name'], how='left').\
                   merge(df_BSN, on=['name'], how='left').\
                   fillna(value=0).\
                   drop(columns=drop_list)
    
    #add some useful columns
    bhbh['tdelay'] = bhbh['GWtime'] + bhbh['BWorldtime']

    bhbh['Mass_max_out'] = bhbh['Mass_1_out']
    bhbh['Mass_max_out'] = bhbh['Mass_max_out'].\
                            where(cond=(bhbh['Mass_max_out'] > bhbh['Mass_0_out']), other=bhbh['Mass_0_out'])

    bhbh['q'] = bhbh['Mass_1_out']/bhbh['Mass_0_out']
    bhbh['q'] = bhbh['q'].\
                where(cond=(bhbh['Mass_1_out'] < bhbh['Mass_0_out']), other=bhbh['Mass_0_out']/bhbh['Mass_1_out'])

    bhbh['Mass_chirp'] = ((bhbh['Mass_0_out'] * bhbh['Mass_1_out'])**(3/5))/((bhbh['Mass_0_out'] + bhbh['Mass_1_out'])**(1/5))
    
    return bhbh

## Map the preprocessing function to the bag

In [8]:
%%time
bag_of_df = bag_2.map(preprocessing_bag_of_thread)

CPU times: user 87.4 ms, sys: 285 µs, total: 87.7 ms
Wall time: 88 ms


## Compute and concat the dfs

In [ ]:
%%time
list_of_df=bag_of_df.compute()
bhbh = dd.multi.concat(list_of_df).compute()

## Alternative way

## Final dataframe

In [ ]:
%%time
head=bhbh.head()
head.to_csv('test_su_test_della_bag_di_bag_aggiornato')

In [ ]:
head